# <span style='color:red'>Project 1</span>

In this project we use dynamic programming to create a trading schedule that maximizes total number of shares traded, under a model of liquidity impact with memory

### Imports

In [ ]:
#from google.colab import drive
import csv
import sys
import scipy.io
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import time

# Task 1
code a dynamic programming algorithm that computes an optimal schedule of trades $(n_0, n_1, \ldots, n_{T-1})$ with the goal of maximizing the total number of traded shares
#### Make sure that your code runs well for a range of values of $\alpha$ and $\pi$
#### Compute the optimal schedule when $\alpha = 0.001$, $\pi = 0.5$, $N = 100000$ and $T = 10$.   Denote this schedule by $(S_0, S_1, \ldots, S_9)$.

In [ ]:
def DP2_vectorized(N, T, alpha, pi):
    # Initialize the DP table R to store total_traded and m values for each t and traded amount.
    R = np.zeros((T, N + 1), dtype=[('total_traded', 'i4'), ('m', 'i4')])

    # Initialize the trace table to keep track of attempted trades at each step.
    trace = np.zeros((T, N + 1), dtype='i4')

    # Precompute trade factor based on alpha and pi for all possible attempt values
    # We assume M can only exist as an integer between 0 and (0.9N+1)
    trade_factor_cache = 1 - alpha * (np.arange(math.ceil(0.9*N) + 1) ** pi)

    # Set initial values in DP table and trace for t = 0
    attempted_trades = np.arange(N + 1)
    m_new = np.ceil(0.1 * 0 + 0.9 * attempted_trades).astype('i4')  # m_new when no shares have been traded yet (at t = 0)
    successfully_traded = np.ceil(trade_factor_cache[m_new] * attempted_trades).astype('i4')
    #the successful trades in this first step are the only trades so far so total_traded=successfully_traded
    total_traded = successfully_traded

    R[0, attempted_trades] = list(zip(total_traded, m_new))
    trace[0, attempted_trades] = attempted_trades

    #for each time period moving forward
    for t in range(1, T):
        # Retrieve total traded and previous m value from table
        prev_total_traded = R[t - 1]['total_traded']
        prev_m = R[t - 1]['m']
        for traded in range(N + 1):
            remaining_shares = N - traded
            max_attempted = remaining_shares  # maximum number of shares we can attempt to trade
            attempted_trades = np.arange(max_attempted + 1)  # all possible attempted amounts

            # Compute m_new on-the-fly instead of using a precomputed cache.
            m_new = np.ceil(0.1 * prev_m[traded] + 0.9 * attempted_trades).astype('i4')
            successfully_traded = np.ceil(trade_factor_cache[m_new] * attempted_trades).astype('i4')  # Calculate successful trades
            total_traded_new = prev_total_traded[traded] + successfully_traded  # Calculate total successfully traded by now
            attempted_trades_new = traded + attempted_trades  # Calculate total attempted amount

            # Update table only if the new value is better (more shares successfully traded)
            #R[t]['total_traded'][attempted_trades_new] at time t is initially a zero array on the first inner loop
            better_indices = total_traded_new > R[t]['total_traded'][attempted_trades_new]
            #update_indices are the indices we of which we have found a better trade schedule up to time t
            update_indices = attempted_trades_new[better_indices]
            R[t]['total_traded'][update_indices] = total_traded_new[better_indices]
            R[t]['m'][update_indices] = m_new[better_indices] #the values of M at t are updated to reflect each new optimal trade schedule up to time t
            trace[t][update_indices] = attempted_trades[better_indices]  # Store attempted trades

    # Find the optimal traded amount
    max_total_traded = R[T - 1]['total_traded'].max() #the maximum number of total shares traded (this must follow the optimal schedule for T periods)
    final_traded = R[T - 1]['total_traded'].argmax() #this is number of shares traded successfully in the last time step of the optimal schedules

    trades = []
    successfully_traded_list = []
    traded = final_traded

    # Backtrack through trace table
    for t in range(T - 1, -1, -1):
        attempted_trade = trace[t][traded]  # Get number of shares attempted at time t
        trades.append(attempted_trade)
        m = R[t][traded]['m']  # Get value of m at time t
        successfully_traded = math.ceil(trade_factor_cache[m] * attempted_trade)  # Calculate successful traded amount
        successfully_traded_list.append(successfully_traded)
        traded -= attempted_trade

    # Reverse lists to get the correct order of trades and successful trades
    trades.reverse()
    successfully_traded_list.reverse()
    return trades, successfully_traded_list, max_total_traded


In [ ]:
#Test the function
import time

start_time = time.time()
attempted_trades, successful_trades, total_trades = DP2_vectorized(N=100000, T=10, alpha=1e-3, pi=0.5)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"DP2_vectorized elapsed time: {elapsed_time} seconds")
print(f"Optimal trade schedule: {attempted_trades}")
print(f"Shares traded: {successful_trades}")
print(f"Total shares traded: {sum(successful_trades)}")

DP2_vectorized elapsed time: 766.6871919631958 seconds
Optimal trade schedule: [9540, 9750, 9874, 9901, 9886, 9851, 9741, 9793, 10471, 11193]
Shares traded: [8657, 8794, 8895, 8917, 8904, 8874, 8780, 8825, 9404, 10014]
Total shares traded: 90064


## Evaluating performance/ Validating results

In [ ]:
M = 0
T = 10
N = 100000
alpha = 1e-3
pi = 0.5
S = np.zeros(T,dtype='i')
n = attempted_trades
print("Planned sales schedule:",n,"; total planned = ",np.sum(n))
total = 0
for i in range(T):
    M = math.ceil(0.1*M + 0.9*n[i])
    S[i] = math.ceil((1 - alpha*M**pi)*n[i])
    total += S[i]
    print('at time %d, M = %d and we trade %d shares' %(i,M,S[i]))
print(S)
print('total sold =', total, "i.e., as a percentage,",100*np.sum(S)/np.sum(n),"of the total planned.")

Planned sales schedule: [9540, 9750, 9874, 9901, 9886, 9851, 9741, 9793, 10471, 11193] ; total planned =  100000
at time 0, M = 8586 and we trade 8657 shares
at time 1, M = 9634 and we trade 8794 shares
at time 2, M = 9850 and we trade 8895 shares
at time 3, M = 9896 and we trade 8917 shares
at time 4, M = 9887 and we trade 8904 shares
at time 5, M = 9855 and we trade 8874 shares
at time 6, M = 9753 and we trade 8780 shares
at time 7, M = 9789 and we trade 8825 shares
at time 8, M = 10403 and we trade 9404 shares
at time 9, M = 11114 and we trade 10014 shares
[ 8657  8794  8895  8917  8904  8874  8780  8825  9404 10014]
total sold = 90064 i.e., as a percentage, 90.064 of the total planned.


As we can see, the our trade schedule has been validated by this testing framework as one that will result in the successful sale of 90064 shares

# Task 2.
Test the effectiveness of this computed schedule using the first 2 hours of each day in the TSLA data </span>
To do so, we divide the first 2 hours of each day into 12 separate intervals of ten minutes each.
Each interval is evaluated as follows.  Suppose that the traded volume in that interval is given by the numbers $(V_0, V_1, \ldots, V_9)$.
Then the interval score we assign to our schedule is given by
$$ \sum_{i = 0}^9 \min\{ S_i, V_i/100 \}.$$
Effectively, this scheme allows us to trade up to a volume of 1% of what the market actually traded.

## Data Preprocessing

In [ ]:
#drive.mount('/content/drive')
#file_path = '/content/drive/MyDrive/Applications Programming/Project 1/TSLA.csv'
file_path = 'TSLA.csv'
df = pd.read_csv(file_path, low_memory=False)

# Dropping unnecessary columns
df = df.iloc[:, :8]
df.columns = df.iloc[2].values

# Keeping only valid rows and dropping NA values
df = df[3:].dropna()

# Converting 'Dates' column to datetime and setting it as index
df['Dates'] = pd.to_datetime(df['Dates'], format='%m/%d/%y %H:%M')
df.set_index('Dates', inplace=True)

# Setting appropriate data types for other columns
columns_to_types = {
    "Open": "float",
    "Close": "float",
    "High": "float",
    "Low": "float",
    "Value": "float",
    "Volume": "int",
    "Number Ticks": "int",
}

for column, dtype in columns_to_types.items():
    df[column] = df[column].astype(dtype)

## Scoring Implementation

In [ ]:
def scoring_10min(n, period, df):
    #This block calculates the score value for each 10 minute interval
    total = 0
    for i in range(10):
        #This part of the loop iterates, minute by minute, across the interval
        timestamp = period + pd.Timedelta(minutes=i)
        if timestamp in df.index:
            #This part of the loop adds the score of the minute that is currently being evaluated
            #The score is the minimum of either the attempted schedule, or 1% of the volume for the minute
            total += min(n[i], df.loc[timestamp, 'Volume']/100)
        else:
            continue
    #Returns the total value for the 10 minute period
    return total


def total(trades):
    #This block calculates the score value for the entire schedule

    #Here, we find the start and end times for the entire period, and then the first half, which is the part we're evaluating
    start_date = df.index.min()
    end_date = df.index.max()
    mid_point = start_date + (end_date - start_date) / 2
    half_data = df[start_date:mid_point]

    total_score = 0
    interval_count = 0
    #This imports the trade schedule
    n = trades
    #This loop iterates across every day in the period we're evaluating
    for day in pd.date_range(start=half_data.index.min(), end=half_data.index.max(), freq='B'):
        #This defines the time period as the first 2 hours of each day, which is the part we're evaluating
        full_time_range = pd.date_range(start=day, end=day + pd.Timedelta(hours=2), freq='T')

        if df.index.isin(full_time_range).sum() == len(full_time_range):
            #Splits the time period into 12 segments of 10 minutes
            time_range = pd.date_range(start=day, periods=12, freq='10T')
            #This part calls the scoring function to add the score for each 10 minute period of the day, and add them to the score for the schedule
            for time_point in time_range:
                total_score += scoring_10min(n, time_point, df)
                interval_count += 1
        else: #skips holidays (MLK, Presidents Day, Good Friday)
            continue
    #Averages and returns the score of the schedule by dividing by the amount of 10 minute periods in the schedule
    average_score = total_score / interval_count if interval_count > 0 else 0
    return average_score

In [ ]:
trades = successful_trades
print('Average score: ', total(trades))

Average score:  8054.838296019905


# Task 3:</span>
Code an algorithm that (approximately) does the following:
#### 1. It approximately enumerates all possible values for $\pi$ between $0.3$ and $0.7$
#### 2. It approximately computes the value of $\pi$ that maximizes the TOTAL SCORE, when $N = 100000$, $T = 10$ and $\alpha = 0.001$.
#### 3. This means that we run the DP algorithm (under the chosen value of $\pi$) and then evaluate as above to compute the TOTAL SCORE.

In [ ]:
N=100000
step = .02
max_ave_score=0
#Initialize max score to 0
max_ave_score=0
for i in np.arange(.3,.7+step,step):
    pi_tmp= round(i,2)
    #Compute the DP schedule for pi_tmp and store it in successfully_traded
    _,successfully_traded,_=DP2_vectorized(N, T, alpha, pi_tmp)
    #Compute the average score for this schedule
    temp_score= total(successfully_traded)
    #Check if the new score is greater than the previous max score and if greater save it as the new max score and save the corresponding Pi
    if temp_score>max_ave_score:
        max_ave_score=temp_score
        max_pi=pi_tmp
        #new max average score so far
        print('current max score: ',max_ave_score,' with pi: ',max_pi)

print('Final best score: ',max_ave_score,' for pi:',max_pi)

current max score:  8056.912922885577  with pi:  0.3
current max score:  8057.126853233835  with pi:  0.32
current max score:  8058.450236318413  with pi:  0.34
current max score:  8058.619390547268  with pi:  0.36
Final best score:  8058.619390547268  for pi: 0.38
